<a href="https://colab.research.google.com/github/WBjunior/Recife_Acidentes-e-infracoes/blob/master/Recife_Acidentes_Infracoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Infrações e acidentes - Recife
## O objetivo dessa análise é obter insights a partir dos dados abertos da CTTU disponível em http://dados.recife.pe.gov.br/. Podemos relacionar dados de acidentes e infrações no perido de 2015 a 2021 para obter informações como as infrações mais recorrentes ao longo dos anos e sua associação ao número de acidentes. Outra questão interessante é o impacto das medidas de lockdown no número de infrações e acidentes 

In [ ]:
import pandas as pd
import numpy as np
import plotly as px


#Base de dados

In [ ]:
acidentes2019= pd.read_csv("http://dados.recife.pe.gov.br/dataset/44087d2d-73b5-4ab3-9bd8-78da7436eed1/resource/3531bafe-d47d-415e-b154-a881081ac76c/download/acidentes-2019.csv",sep=';')
acidentes2020= pd.read_csv("http://dados.recife.pe.gov.br/dataset/44087d2d-73b5-4ab3-9bd8-78da7436eed1/resource/fc1c8460-0406-4fff-b51a-e79205d1f1ab/download/acidentes_2020-novo.csv",sep=';')
acidentes2021= pd.read_csv("http://dados.recife.pe.gov.br/dataset/44087d2d-73b5-4ab3-9bd8-78da7436eed1/resource/2caa8f41-ccd9-4ea5-906d-f66017d6e107/download/acidentes2021.csv",sep=';')
infracoes2019= pd.read_csv("http://dados.recife.pe.gov.br/dataset/6399f689-f1a7-453b-b839-413bd665c355/resource/5369f3fd-ce5f-4409-b9b1-19ff90c4f6ac/download/relatorio-de-multas-implantadas-em-2019.csv",sep=';')
infracoes2020= pd.read_csv("http://dados.recife.pe.gov.br/dataset/6399f689-f1a7-453b-b839-413bd665c355/resource/b065a300-7a68-4d18-8d32-7eb160a19208/download/infracoes2020completo.csv",sep=';')
infracoes2021= pd.read_csv("http://dados.recife.pe.gov.br/dataset/6399f689-f1a7-453b-b839-413bd665c355/resource/f7d76884-e034-41e1-8837-9d8ceb9f22d7/download/infracoes2021completonovo.csv",sep=';')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
acidentes2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2380 entries, 0 to 2379
Data columns (total 37 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   data                       2380 non-null   object
 1   hora                       2380 non-null   object
 2   natureza_acidente          2380 non-null   object
 3   situacao                   2380 non-null   object
 4   bairro                     2375 non-null   object
 5   endereco                   2376 non-null   object
 6   numero                     1301 non-null   object
 7   detalhe_endereco_acidente  909 non-null    object
 8   complemento                2200 non-null   object
 9   bairro_cruzamento          2375 non-null   object
 10  num_semaforo               611 non-null    object
 11  sentido_via                1513 non-null   object
 12  tipo                       2322 non-null   object
 13  auto                       2380 non-null   int64 
 14  moto    

In [ ]:
infracoes2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598556 entries, 0 to 598555
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   datainfracao       598556 non-null  object
 1   horainfracao       598556 non-null  object
 2   dataimplantacao    598556 non-null  object
 3   agenteequipamento  598556 non-null  object
 4   infracao           598556 non-null  int64 
 5   descricaoinfracao  598556 non-null  object
 6   amparolegal        598556 non-null  object
 7   localcometimento   598556 non-null  object
dtypes: int64(1), object(7)
memory usage: 36.5+ MB


## Filtros

In [ ]:
filtro1 = ['data','hora','natureza_acidente','bairro','endereco','tipo','auto','moto','ciclom','ciclista',
           'pedestre','vitimas','vitimasfatais','tempo_clima','condicao_via','conservacao_via','velocidade_max_via','mao_direcao']
filtro2 = ['datainfracao','horainfracao','descricaoinfracao','agenteequipamento','localcometimento']

In [ ]:
acidentes2019 = acidentes2019.rename({'DATA': 'data'}, axis = 1)

In [ ]:
acidentes2019filter= acidentes2019[filtro1]
acidentes2020filter= acidentes2020[filtro1]
acidentes2021filter= acidentes2021[filtro1]
infracoes2019filter= infracoes2019[filtro2]
infracoes2020filter= infracoes2020[filtro2]
infracoes2021filter= infracoes2021[filtro2]

In [ ]:
acidentes = pd.concat([acidentes2019filter, acidentes2020filter, acidentes2021filter], axis=0)
acidentes['data'] = acidentes['data'].astype('datetime64[ms]')
infracoes = pd.concat([infracoes2019filter,infracoes2020filter,infracoes2021filter],axis=0)
infracoes['datainfracao'] = infracoes['datainfracao'].astype('datetime64[ms]')

In [ ]:
acidentes.describe(include=[object])

,hora,natureza_acidente,bairro,endereco,tipo,tempo_clima,condicao_via,conservacao_via,velocidade_max_via,mao_direcao
count,18518,18529,18347,18464,18476,14879,14830,14606,4609,14569
unique,1508,6,94,1648,27,3,4,4,26,3
top,14:45:00,SEM VÍTIMA,BOA VIAGEM,AV GOVERNADOR AGAMENON MAGALHAES,ABALROAMENTO LONGITUDINAL,Bom,Seca,Perfeito estado,60 km/h,Dupla
freq,38,12653,2297,843,4836,13233,13294,13688,2529,8445


In [ ]:
acidentes.describe(exclude=[object],datetime_is_numeric=True)

,data,auto,moto,ciclom,ciclista,pedestre,vitimas,vitimasfatais
count,18534,17004.000000,6976.000000,2447.000000,2756.000000,2790.000000,18493.000000,2426.000000
mean,2019-12-16 05:43:43.373259776,1.466302,0.928899,0.035145,0.194122,0.217921,0.367977,0.028854
min,2019-01-01 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2019-05-27 00:00:00,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2019-10-18 12:00:00,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2020-02-29 00:00:00,2.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,2021-12-31 00:00:00,9.000000,4.000000,1.000000,3.000000,5.000000,11.000000,1.000000
std,NaN,0.678507,0.447491,0.184184,0.403768,0.451087,0.602150,0.167431


In [ ]:
infracoes.describe(include=[object])

,horainfracao,descricaoinfracao,agenteequipamento,localcometimento
count,2173808,2173808,2173808,2173801
unique,60683,301,8,247357
top,09:30:00,Transitar em velocidade superior à máxima perm...,Código 8 - AUTOS NO TALÃO ELETRÔNICO,"AV. ENG. ABDIAS DE CARVALHO, ENTRE OS N. 1785 ..."
freq,4110,645670,1043634,46618


In [ ]:
infracoes.describe(exclude=[object],datetime_is_numeric=True)

,datainfracao
count,2173805
mean,2020-05-17 15:38:00.579353600
min,2019-01-01 00:00:00
25%,2019-09-04 00:00:00
50%,2020-03-17 00:00:00
75%,2021-02-02 00:00:00
max,2021-12-31 00:00:00


In [ ]:

import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=3)

x0 = np.array(acidentes[(acidentes['data']<'2020-01-01')]['data'].value_counts())
x1 = np.array(acidentes[(acidentes['data']>='2020-01-01')&(acidentes['data']<'2021-01-01')]['data'].value_counts())
x2 = np.array(acidentes[(acidentes['data']>='2021-01-01')]['data'].value_counts())

trace0 = go.Histogram(x=x0,name='Acidentes 2019',nbinsx=20)
trace1 = go.Histogram(x=x1,name='Acidentes 2020',nbinsx=20)
trace2 = go.Histogram(x=x2,name='Acidentes 2021',nbinsx=20)

fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig.append_trace(trace2, 1, 3)
fig.update_layout(barmode='stack',height=800,width=1000,yaxis_title='Frequência')
fig.update_xaxes(title_text="N° de infrações por dia", row=1, col=2)
fig.show()

In [ ]:
x0 = np.array(infracoes[(infracoes['datainfracao']<'2020-01-01')]['datainfracao'].value_counts())
x1 = np.array(infracoes[(infracoes['datainfracao']>='2020-01-01')&(infracoes['datainfracao']<'2021-01-01')]['datainfracao'].value_counts())
x2 = np.array(infracoes[(infracoes['datainfracao']>='2021-01-01')&(infracoes['datainfracao']<'2022-01-01')]['datainfracao'].value_counts())
fig = make_subplots(rows=1, cols=3)

trace0=go.Histogram(x=x0,name='Infrações 2019',nbinsx=20)
trace1=go.Histogram(x=x1,name='Infrações 2020',nbinsx=20)
trace2=go.Histogram(x=x2,name='Infrações 2021',nbinsx=20)  
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig.append_trace(trace2, 1, 3)

fig.update_layout(barmode='stack',height=800,width=1000,yaxis_title='Frequência')
fig.update_xaxes(title_text="N° de infrações por dia", row=1, col=2)
fig.show()

## 1- Qual são as infrações mais frequentes na avenida que apresenta o maior número de acidentes?



### Vamos filtrar os acidentes e infrações que ocorreram na AV GOVERNADOR AGAMENON MAGALHAES que foi o local onde ocorreu o maior numero de acidentes

In [ ]:
df = infracoes
df['Agamenon']= infracoes.localcometimento.str.contains("AGAMENON MAGALHAES")
infracoesagamenon = df.query("Agamenon==True")

In [ ]:
infracoesagamenon['descricaoinfracao'].value_counts().nlargest(n=20)

Transitar em velocidade superior à máxima permitida para o local em até 20% (vinte por cento).          37286
Dirigir o veículo segurando o telefone celular.                                                         17063
Executar operação de conversão à esquerda em locais proibidos pela sinalização.                         11980
Dirigir o veículo manuseando o telefone celular.                                                         5982
Dirigir sem atenção ou sem os cuidados indispensáveis à Segurança.                                       4937
Avançar o sinal vermelho do semáforo.                                                                    4400
Dirigir o veículo utilizando-se de telefone celular.                                                     3259
Transitar na faixa ou via exclusiva regulam.  p/  transp.  públ.  coletivo passag.                       3219
Transitar em velocidade superior à máxima permitida para o local em mais de 20% (vinte por cento) at     3199
Dirigir o 

In [ ]:
df = acidentes
df['Agamenon']= df.endereco.str.contains("AGAMENON MAGALHAES")
acidentessagamenon = df.query("Agamenon==True")

In [ ]:
acidentessagamenon['tipo'].value_counts()

COLISÃO TRASEIRA             297
ABALROAMENTO LONGITUDINAL    213
COLISÃO                      124
ABALROAMENTO TRANSVERSAL      56
COLISÃO LATERAL               31
ATROPELAMENTO                 21
COLISÃO TRANSVERSAL           20
COLISÃO COM CICLISTA          19
COLISÃO FRONTAL               13
ATROPELAMENTO DE PESSOA       10
CHOQUE                         9
SEMÁFORO                       6
CAPOTAMENTO                    6
TOMBAMENTO                     6
ENGAVETAMENTO                  6
CHOQUE OBJETO FIXO             5
CHOQUE VEÍCULO PARADO          4
QUEDA                          3
ACID. DE PERCURSO              1
Name: tipo, dtype: int64

### As infrações mais frequentes são Transitar em velocidade superior à máxima permitida para o local em até 20% (vinte por cento), dirigir o veículo segurando o telefone celular e executar operação de conversão à esquerda em locais proibidos pela sinalização. Podemos ir além e calcular a correlação entre essas infrações e a quantidade de acidentes diárias na Av. Gov. Agamenon Magalhães. 

In [ ]:
# Crie uma lista de descrições de infração
descricoes_infracao  = {'Transitar em velocidade superior à máxima permitida para o local em até 20% (vinte por cento).':'velocidade < 20%',
                       'Dirigir o veículo segurando o telefone celular.':'celular',
                       'Executar operação de conversão à esquerda em locais proibidos pela sinalização.':'Conversão proibida',
                       'Dirigir sem atenção ou sem os cuidados indispensáveis à Segurança.':'Digirir sem atenção',
                       'Avançar o sinal vermelho do semáforo.':'Avançar o sinal',
                       'Transitar na faixa ou via exclusiva regulam.  p/  transp.  públ.  coletivo passag.':'Transitar faixa onibus',
                       'Transitar em velocidade superior à máxima permitida para o local em mais de 20% (vinte por cento) at':'Velocidade >20%',
                       'Dirigir o veículo usando calçado que não se firme nos pés ou que comprometa a utilização dos pedais.':'Calçado inadequado',
                       'Executar operação de conversão à direita em locais proibidos pela sinalização.':'Conversão a direita',
                       'Estacionar o veículo no passeio.':'Estacionamento proibido',                                                                         
                       'Parar o veículo na área de cruzamento de vias, prejudicando a circulação de veículos e pedestres.':'Parar no cruzamento',        
                       'Deixar o passageiro de usar o cinto de segurança, conforme o Art. 65, do CTB.':'Cinto de segurança',                          
                       'Quando o veículo estiver em movimento deixar de manter acesa a luz baixa durante a noite.':'Luz baixa',               
                       'Deixar de deslocar, com antecedência, o veículo para a faixa mais à direita, quando for manobrar.':'Manobra',       
                       'Parar o veículo sobre a faixa de pedestre na mudança de sinal luminoso.':'Parar na faixa',                                  
                       'Deixar de indicar com antecedência, mediante gesto regulamentar de braço ou luz indicadora de direçã':'Deixar de indicar',     
                       'Dirigir o veículo com apenas uma das mãos, exceto quando permitido pelo CTB.':'Dirigir com uma mão',                             
                       'Executar operação de retorno com prejuízo da livre circulação ou da segurança, ainda que em locais p':'Retorno inadequado'}


# Crie uma nova tabela que contém apenas as linhas que correspondem às descrições de infração acima
acidentesav = acidentessagamenon['data'].value_counts()
infracoesav = infracoesagamenon['datainfracao'].value_counts()
df = pd.DataFrame()
df = pd.merge(df, infracoesav, how = 'outer', left_index=True, right_index=True)
df = df.rename(columns={'datainfracao': 'infrações'})
df = pd.merge(df,acidentesav, how = 'outer', left_index=True, right_index=True)
df = df.rename(columns={'data': 'Acidentes'})
for keys in descricoes_infracao.keys():
  df = pd.merge(df, infracoesagamenon[(infracoesagamenon['descricaoinfracao']==keys)]['datainfracao'].value_counts(), how = 'outer', left_index=True, right_index=True)
  df = df.rename(columns={'datainfracao': descricoes_infracao[keys]})
  df = df.fillna(0)



In [ ]:
df.corr(method ='spearman')

,infrações,Acidentes,velocidade < 20%,celular,Conversão proibida,Digirir sem atenção,Avançar o sinal,Transitar faixa onibus,Velocidade >20%,Calçado inadequado,Conversão a direita,Estacionamento proibido,Parar no cruzamento,Cinto de segurança,Luz baixa,Manobra,Parar na faixa,Deixar de indicar,Dirigir com uma mão,Retorno inadequado
infrações,1.000000,0.286833,0.114441,0.709814,0.514420,0.639454,0.634053,0.233810,0.077780,0.586244,0.249073,0.381189,0.565098,0.596883,0.614583,0.428741,0.556101,0.494277,0.586983,0.118422
Acidentes,0.286833,1.000000,0.023953,0.261834,0.033349,0.389942,0.295321,-0.141969,-0.076266,0.277651,-0.176292,0.193377,0.311862,0.177174,0.276368,0.234831,0.252624,0.068997,0.276417,-0.113582
velocidade < 20%,0.114441,0.023953,1.000000,-0.323121,-0.031597,-0.192822,-0.187237,-0.120796,0.707660,-0.299309,-0.225535,-0.012989,-0.294502,-0.186299,-0.192957,-0.121816,-0.235242,-0.165163,-0.214913,-0.123212
celular,0.709814,0.261834,-0.323121,1.000000,0.381922,0.656308,0.601914,0.201485,-0.270540,0.738647,0.411161,0.309782,0.644283,0.650486,0.648155,0.444100,0.596778,0.510435,0.628567,0.146413
Conversão proibida,0.514420,0.033349,-0.031597,0.381922,1.000000,0.215205,0.313724,0.305991,0.016285,0.236891,0.528584,0.138982,0.282929,0.360940,0.234361,0.182047,0.286519,0.348259,0.202037,0.229075
Digirir sem atenção,0.639454,0.389942,-0.192822,0.656308,0.215205,1.000000,0.743100,-0.015839,-0.224886,0.646288,0.029146,0.261925,0.610657,0.588309,0.596138,0.528486,0.639184,0.375094,0.617384,-0.033554
Avançar o sinal,0.634053,0.295321,-0.187237,0.601914,0.313724,0.743100,1.000000,0.035684,-0.212009,0.559815,0.104683,0.257808,0.569911,0.617792,0.611399,0.509779,0.586227,0.493398,0.631989,-0.067635
Transitar faixa onibus,0.233810,-0.141969,-0.120796,0.201485,0.305991,-0.015839,0.035684,1.000000,-0.062185,0.145941,0.426285,0.021980,0.100131,0.142162,0.088536,0.003019,0.085160,0.212647,0.085754,0.226556
Velocidade >20%,0.077780,-0.076266,0.707660,-0.270540,0.016285,-0.224886,-0.212009,-0.062185,1.000000,-0.246771,-0.109482,-0.031691,-0.234512,-0.179577,-0.190652,-0.156928,-0.202473,-0.128345,-0.202491,-0.059698
Calçado inadequado,0.586244,0.277651,-0.299309,0.738647,0.236891,0.646288,0.559815,0.145941,-0.246771,1.000000,0.281868,0.267857,0.532631,0.575348,0.584251,0.407135,0.525352,0.375430,0.553799,0.097904


### Dado o resultado,existe correlação fraca entre os dados de acidentes e infrações. Em relação as infrações mais frequentes Digirir sem atenção, a infração por andar utilizando o celular, Parar na faixa de pedestre, Deixar de deslocar, com antecedência, o veículo para a faixa mais à direita, quando for manobrar,Deixar de manter a luz baixa a noite, Parar no cruzamento, Calçado inadequado e Avançar o sinal também apresentaram correlação fraca com os acidentes.    




## 2 - Tempo , condição da via e conservação são variáveis significativas na ocorrência de acidentes ?



### Para responder essa pergunta podemos usar o teste chi quadrado para verificar se existe evidencia estatistica de independencia das variáveis tempo_clima, condicao_via e conservacao_via.

In [ ]:
acidentesnotnan= acidentes.dropna()
from scipy.stats import chi2_contingency
def testechi2(ds1,ds2) :
  cross = pd.crosstab(ds1, ds2,)
  stat, p, dof, expected = chi2_contingency(cross)
  alpha = 0.05
  print("p value is " + str(p))
  if p <= alpha:  
    print('Rejeita H0')
  else:
    print('Nao Rejeita H0')

In [ ]:
print('Teste  χ2 tempo_clima e conservacao_via :')
testechi2(acidentesnotnan['tempo_clima'],acidentesnotnan['conservacao_via'])
print('Teste  χ2 tempo_clima e condicao_via :')
testechi2(acidentesnotnan['tempo_clima'],acidentesnotnan['condicao_via'])
print('Teste  χ2 condicao_via e conservacao_via :')
testechi2(acidentesnotnan['condicao_via'], acidentesnotnan['conservacao_via'])

Teste  χ2 tempo_clima e conservacao_via :
p value is 0.9713184617731951
Nao Rejeita H0
Teste  χ2 tempo_clima e condicao_via :
p value is 2.253983874947682e-104
Rejeita H0
Teste  χ2 condicao_via e conservacao_via :
p value is 0.7955149139981361
Nao Rejeita H0


### De acordo com os resultados do teste χ2 podemos dizer que não existe diferença estatisticamente significante entre tempo_clima e condicao_via, portanto elas podem ser consideradas como dependentes.



##3- Como as restrições de deslocamento da pandemia afetou o número de acidentes e infrações em 2020?



### Ao observar os histogramas da quantidade diária de acidentes e infrações no ano de 2020 podemos ver que existe uma assimetria acentuada e particulamente no histograma de infrações é possivel enxergar um comportamento bimodal o que pode indicar 2 comportamentos diferentes no periodo de pandemia. Vamos avaliar o numero de infrações por dia no primeiro e segunto semestre de 2020 e comparar com 2019:  

In [ ]:
infracoes20201 = infracoes[(infracoes['datainfracao']>='2020-01-01')&(infracoes['datainfracao']<'2020-04-01')]['datainfracao'].value_counts().sort_values()
infracoes20202 = infracoes[(infracoes['datainfracao']>='2020-04-01')&(infracoes['datainfracao']<'2020-07-01')]['datainfracao'].value_counts().sort_values()
infracoes20203 = infracoes[(infracoes['datainfracao']>='2020-07-01')&(infracoes['datainfracao']<'2020-10-01')]['datainfracao'].value_counts().sort_values()
infracoes20204 = infracoes[(infracoes['datainfracao']>='2020-10-01')&(infracoes['datainfracao']<'2021-01-01')]['datainfracao'].value_counts().sort_values()
infracoes20191 = infracoes[(infracoes['datainfracao']>='2019-01-01')&(infracoes['datainfracao']<'2019-04-01')]['datainfracao'].value_counts().sort_values()
infracoes20192 = infracoes[(infracoes['datainfracao']>='2019-04-01')&(infracoes['datainfracao']<'2019-07-01')]['datainfracao'].value_counts().sort_values()
infracoes20193 = infracoes[(infracoes['datainfracao']>='2019-07-01')&(infracoes['datainfracao']<'2019-10-01')]['datainfracao'].value_counts().sort_values()
infracoes20194 = infracoes[(infracoes['datainfracao']>='2019-10-01')&(infracoes['datainfracao']<'2020-01-01')]['datainfracao'].value_counts().sort_values()

In [ ]:
df1 = infracoes20201.to_frame(name='values')
df1['freq'] = np.linspace(0, 1, infracoes20201.size)
df2 = infracoes20202.to_frame(name='values')
df2['freq'] = np.linspace(0, 1, infracoes20202.size)
df3 = infracoes20203.to_frame(name='values')
df3['freq'] = np.linspace(0, 1, infracoes20203.size)
df4 = infracoes20204.to_frame(name='values')
df4['freq'] = np.linspace(0, 1, infracoes20204.size)


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df1['values'], y=df1['freq'],name='Jan-Fev-Mar',mode="markers"))
fig.add_trace(go.Scatter(x=df2['values'], y=df2['freq'],name='Abr-Mai-Jun',mode="markers"))
fig.add_trace(go.Scatter(x=df3['values'], y=df3['freq'],name='Jul-Ago-Set',mode="markers"))
fig.add_trace(go.Scatter(x=df4['values'], y=df4['freq'],name='Out-Nov-Dez',mode="markers"))
fig.update_layout(title='Infrações 2020',height=800,width=1000,yaxis_title='ECDF',
xaxis_title='N° de infrações por dia')
fig.show()

In [ ]:
df1 = infracoes20191.to_frame(name='values')
df1['freq'] = np.linspace(0, 1, infracoes20191.size)
df2 = infracoes20192.to_frame(name='values')
df2['freq'] = np.linspace(0, 1, infracoes20192.size)
df3 = infracoes20193.to_frame(name='values')
df3['freq'] = np.linspace(0, 1, infracoes20193.size)
df4 = infracoes20194.to_frame(name='values')
df4['freq'] = np.linspace(0, 1, infracoes20194.size)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df1['values'], y=df1['freq'],name='Jan-Fev-Mar',mode="markers"))
fig.add_trace(go.Scatter(x=df2['values'], y=df2['freq'],name='Abr-Mai-Jun',mode="markers"))
fig.add_trace(go.Scatter(x=df3['values'], y=df3['freq'],name='Jul-Ago-Set',mode="markers"))
fig.add_trace(go.Scatter(x=df4['values'], y=df4['freq'],name='Out-Nov-Dez',mode="markers"))
fig.update_layout(title='Infrações 2019',height=800,width=1000,yaxis_title='ECDF',
xaxis_title='N° de infrações por dia')
fig.show()

### Ao visualizar os graficos acima, podemos retirar as seguintes informaçõe sobre o ano de 2020:
### No trimestre Abril, Maio e Junho (auge da pandemia e medidas de restrição de mobilização) 75% do número de infrações registrados por dia é menor ou igual a  1113 o que difere do observado em 2019(2334). 
### Em relação ao último trimestre, temos 75% do número de infrações por dia menor ou igual a 3353 em 2020 e 3333 em 2019. A partir das informações fornecidas, é possível concluir que houve uma significativa queda no número de infrações registradas durante o auge da pandemia em 2020 em comparação com o mesmo período do ano anterior. Isso pode ser atribuído às medidas de restrição de mobilidade implementadas para conter a propagação da COVID-19. Além disso, também é possível ver que o número de infrações no último trimestre de 2020 foi semelhante ao do mesmo período do ano anterior, sugerindo que as medidas de restrição foram relaxadas e o tráfego de veículos voltou ao nível anterior à pandemia.



##4 - Existe têndencia de decrescimo no número de infrações e acidentes?

In [ ]:
#frenquencia diária
seriesfreqacidentes =  acidentes['data'].value_counts()
seriesfreqinfracoes =  infracoes['datainfracao'].value_counts()

In [ ]:
dfacidpordia = seriesfreqacidentes.to_frame()
dfacidpordia = dfacidpordia.reset_index()
dfacidpordia = dfacidpordia.rename(columns={'data': 'acidentes','index':'data'})
dfacidpordia = dfacidpordia.sort_values(by='data')
dfinfrapordia = seriesfreqinfracoes.to_frame()
dfinfrapordia = dfinfrapordia.reset_index()
dfinfrapordia = dfinfrapordia.rename(columns={'datainfracao': 'infracoes','index':'data'})
dfinfrapordia = dfinfrapordia.sort_values(by='data')

In [ ]:
#importar pacotes e setar configurações de plots
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose

dfinfrapordia['data'] = pd.to_datetime(dfinfrapordia['data'])
dfinfrapordia.set_index('data',inplace=True) 
result = seasonal_decompose(dfinfrapordia['infracoes'])
fig = make_subplots(
            rows=4, cols=1,
            subplot_titles=["Série", "Tendência", "Sazonalidade", "Residuos"])
fig.add_trace(go.Scatter(x=result.seasonal.index, y=result.observed, mode='lines'),row=1, col=1,)
fig.add_trace(go.Scatter(x=result.trend.index, y=result.trend, mode='lines'),row=2, col=1,)
fig.add_trace(go.Scatter(x=result.seasonal.index, y=result.seasonal, mode='lines'),row=3, col=1,)
fig.add_trace(go.Scatter(x=result.resid.index, y=result.resid, mode='lines'),row=4, col=1,
)

fig.show()

In [ ]:
dfacidpordia['data'] = pd.to_datetime(dfacidpordia['data'])
dfacidpordia.set_index('data',inplace=True) 
result = seasonal_decompose(dfacidpordia['acidentes'],period=1)
fig = make_subplots(rows=4, cols=1,subplot_titles=["Série", "Tendência", "Sazonalidade", "Residuos"])
fig.add_trace(go.Scatter(x=result.seasonal.index, y=result.observed, mode='lines'),row=1, col=1,)
fig.add_trace(go.Scatter(x=result.trend.index, y=result.trend, mode='lines'),row=2, col=1,)
fig.add_trace(go.Scatter(x=result.seasonal.index, y=result.seasonal, mode='lines'),row=3, col=1,)
fig.add_trace(go.Scatter(x=result.resid.index, y=result.resid, mode='lines'),row=4, col=1,
)

fig.show()

### Podemos vizualizar, através do gráfico de tendencia de acidentes e infrações, que existe um vale que inicia em Março de 2020, em relação as infrações a partir de Junho de 2020 existe um pequeno aumento e permanece desse nivel com flutuações. O nível de acidentes permanece de Março até Dezembro de 2021. É possível que essa tendência de queda nas infrações e acidentes a partir de março de 2020 seja devido às medidas de distanciamento social e lockdowns implementadas em muitos países devido à pandemia de COVID-19. Com menos pessoas nas ruas e menos tráfego de veículos, pode haver menos oportunidades para infrações e acidentes. O aumento leve nas infrações a partir de junho de 2020 pode ser devido ao relaxamento das medidas de distanciamento social e ao aumento do tráfego de veículos. A estabilidade no nível de acidentes até dezembro de 2021 pode ser devido ao equilíbrio entre o aumento do tráfego e as medidas de segurança ainda em vigor.

## 5 - Quais são os veiculos mais importantes em relação ao numero de acidentes com vítimas?

In [ ]:
veiculos = ['auto', 'moto', 'ciclom', 'ciclista', 'pedestre']
acidcomvit = []
acidcomvitf = []
for itens in veiculos:
  y = acidentes[(acidentes[itens]>0)]['vitimas'].sum()
  x = itens
  acidcomvit.append([x,y])
x1,y1=zip(*acidcomvit)
for itens in veiculos:
  y = acidentes[(acidentes[itens]>0)]['vitimasfatais'].sum()
  x = itens
  acidcomvitf.append([x,y])
x2,y2=zip(*acidcomvitf)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=x1,y=y1,mode='markers',name='Vítimas'))
fig.add_trace(go.Scatter(x=x2,y=y2,mode='markers',name='Vítimas fatais'))
fig.update_layout(title='Nº de Vítimas',height=600,width=800,yaxis_title='N° de vitimas',
xaxis_title='Tipo de veículo')
fig.update_traces(mode='markers', marker_line_width=2, marker_size=10)
fig.show()

Verificamos que os acidentes com maior número de vitimas são os que envolvem carros e motos, seguidos pelos acidentes envolvendo pedestres, ciclistas e ciclomotos. Vamos explorar melhor os acidentes envolvendo carros e motos:

In [ ]:

acidentescarro = acidentes[(acidentes['auto']>0)&(acidentes['moto']==0)]
acidentesmoto = acidentes[(acidentes['moto']>0)&(acidentes['auto']==0)]
acidentescarroemoto = acidentes [(acidentes['auto']>0)&(acidentes['moto']>0)]


In [ ]:
acidentescarro.describe(include=[object])

,hora,natureza_acidente,bairro,endereco,tipo,tempo_clima,condicao_via,conservacao_via,velocidade_max_via,mao_direcao
count,683,683,682,681,650,464,463,459,159,453
unique,516,4,73,286,14,3,4,3,13,2
top,10:13:00,COM VÍTIMA,BOA VIAGEM,AV GOVERNADOR AGAMENON MAGALHAES,COLISÃO TRASEIRA,Bom,Seca,Perfeito estado,60 km/h,Dupla
freq,5,339,79,43,132,394,389,439,80,260


In [ ]:
acidentesmoto.describe(include=[object])

,hora,natureza_acidente,bairro,endereco,tipo,tempo_clima,condicao_via,conservacao_via,velocidade_max_via,mao_direcao
count,381,381,381,381,377,325,324,317,128,318
unique,322,4,71,176,14,3,3,4,5,2
top,10:03:00,COM VÍTIMA,BOA VIAGEM,AV NORTE,COLISÃO LATERAL,Bom,Seca,Perfeito estado,60 km/h,Dupla
freq,3,346,25,18,75,274,271,296,78,213


In [ ]:
acidentescarroemoto.describe(include=[object])

,hora,natureza_acidente,bairro,endereco,tipo,tempo_clima,condicao_via,conservacao_via,velocidade_max_via,mao_direcao
count,4687,4690,4675,4684,4678,3891,3887,3833,1241,3787
unique,1163,3,93,794,17,3,4,4,10,3
top,12:15:00,COM VÍTIMA,BOA VIAGEM,AV GOVERNADOR AGAMENON MAGALHAES,ABALROAMENTO LONGITUDINAL,Bom,Seca,Perfeito estado,60 km/h,Dupla
freq,14,3236,529,190,1137,3502,3533,3610,664,2444


#Infrações e acidentes - Recife
## O objetivo dessa análise é obter insights a partir dos dados abertos da CTTU disponível em http://dados.recife.pe.gov.br/. Podemos relacionar dados de acidentes e infrações no perido de 2015 a 2021 para obter informações como as infrações mais recorrentes ao longo dos anos e sua associação ao número de acidentes. Outra questão interessante é o impacto das medidas de lockdown no número de infrações e acidentes 

In [ ]:
import pandas as pd
import numpy as np
import plotly as px


#Base de dados

In [ ]:
acidentes2019= pd.read_csv("http://dados.recife.pe.gov.br/dataset/44087d2d-73b5-4ab3-9bd8-78da7436eed1/resource/3531bafe-d47d-415e-b154-a881081ac76c/download/acidentes-2019.csv",sep=';')
acidentes2020= pd.read_csv("http://dados.recife.pe.gov.br/dataset/44087d2d-73b5-4ab3-9bd8-78da7436eed1/resource/fc1c8460-0406-4fff-b51a-e79205d1f1ab/download/acidentes_2020-novo.csv",sep=';')
acidentes2021= pd.read_csv("http://dados.recife.pe.gov.br/dataset/44087d2d-73b5-4ab3-9bd8-78da7436eed1/resource/2caa8f41-ccd9-4ea5-906d-f66017d6e107/download/acidentes2021.csv",sep=';')
infracoes2019= pd.read_csv("http://dados.recife.pe.gov.br/dataset/6399f689-f1a7-453b-b839-413bd665c355/resource/5369f3fd-ce5f-4409-b9b1-19ff90c4f6ac/download/relatorio-de-multas-implantadas-em-2019.csv",sep=';')
infracoes2020= pd.read_csv("http://dados.recife.pe.gov.br/dataset/6399f689-f1a7-453b-b839-413bd665c355/resource/b065a300-7a68-4d18-8d32-7eb160a19208/download/infracoes2020completo.csv",sep=';')
infracoes2021= pd.read_csv("http://dados.recife.pe.gov.br/dataset/6399f689-f1a7-453b-b839-413bd665c355/resource/f7d76884-e034-41e1-8837-9d8ceb9f22d7/download/infracoes2021completonovo.csv",sep=';')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
acidentes2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2380 entries, 0 to 2379
Data columns (total 37 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   data                       2380 non-null   object
 1   hora                       2380 non-null   object
 2   natureza_acidente          2380 non-null   object
 3   situacao                   2380 non-null   object
 4   bairro                     2375 non-null   object
 5   endereco                   2376 non-null   object
 6   numero                     1301 non-null   object
 7   detalhe_endereco_acidente  909 non-null    object
 8   complemento                2200 non-null   object
 9   bairro_cruzamento          2375 non-null   object
 10  num_semaforo               611 non-null    object
 11  sentido_via                1513 non-null   object
 12  tipo                       2322 non-null   object
 13  auto                       2380 non-null   int64 
 14  moto    

In [ ]:
infracoes2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598556 entries, 0 to 598555
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   datainfracao       598556 non-null  object
 1   horainfracao       598556 non-null  object
 2   dataimplantacao    598556 non-null  object
 3   agenteequipamento  598556 non-null  object
 4   infracao           598556 non-null  int64 
 5   descricaoinfracao  598556 non-null  object
 6   amparolegal        598556 non-null  object
 7   localcometimento   598556 non-null  object
dtypes: int64(1), object(7)
memory usage: 36.5+ MB


## Filtros

In [ ]:
filtro1 = ['data','hora','natureza_acidente','bairro','endereco','tipo','auto','moto','ciclom','ciclista',
           'pedestre','vitimas','vitimasfatais','tempo_clima','condicao_via','conservacao_via','velocidade_max_via','mao_direcao']
filtro2 = ['datainfracao','horainfracao','descricaoinfracao','agenteequipamento','localcometimento']

In [ ]:
acidentes2019 = acidentes2019.rename({'DATA': 'data'}, axis = 1)

In [ ]:
acidentes2019filter= acidentes2019[filtro1]
acidentes2020filter= acidentes2020[filtro1]
acidentes2021filter= acidentes2021[filtro1]
infracoes2019filter= infracoes2019[filtro2]
infracoes2020filter= infracoes2020[filtro2]
infracoes2021filter= infracoes2021[filtro2]

In [ ]:
acidentes = pd.concat([acidentes2019filter, acidentes2020filter, acidentes2021filter], axis=0)
acidentes['data'] = acidentes['data'].astype('datetime64[ms]')
infracoes = pd.concat([infracoes2019filter,infracoes2020filter,infracoes2021filter],axis=0)
infracoes['datainfracao'] = infracoes['datainfracao'].astype('datetime64[ms]')

In [ ]:
acidentes.describe(include=[object])

,hora,natureza_acidente,bairro,endereco,tipo,tempo_clima,condicao_via,conservacao_via,velocidade_max_via,mao_direcao
count,18518,18529,18347,18464,18476,14879,14830,14606,4609,14569
unique,1508,6,94,1648,27,3,4,4,26,3
top,14:45:00,SEM VÍTIMA,BOA VIAGEM,AV GOVERNADOR AGAMENON MAGALHAES,ABALROAMENTO LONGITUDINAL,Bom,Seca,Perfeito estado,60 km/h,Dupla
freq,38,12653,2297,843,4836,13233,13294,13688,2529,8445


In [ ]:
acidentes.describe(exclude=[object],datetime_is_numeric=True)

,data,auto,moto,ciclom,ciclista,pedestre,vitimas,vitimasfatais
count,18534,17004.000000,6976.000000,2447.000000,2756.000000,2790.000000,18493.000000,2426.000000
mean,2019-12-16 05:43:43.373259776,1.466302,0.928899,0.035145,0.194122,0.217921,0.367977,0.028854
min,2019-01-01 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2019-05-27 00:00:00,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2019-10-18 12:00:00,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2020-02-29 00:00:00,2.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,2021-12-31 00:00:00,9.000000,4.000000,1.000000,3.000000,5.000000,11.000000,1.000000
std,NaN,0.678507,0.447491,0.184184,0.403768,0.451087,0.602150,0.167431


In [ ]:
infracoes.describe(include=[object])

,horainfracao,descricaoinfracao,agenteequipamento,localcometimento
count,2173808,2173808,2173808,2173801
unique,60683,301,8,247357
top,09:30:00,Transitar em velocidade superior à máxima perm...,Código 8 - AUTOS NO TALÃO ELETRÔNICO,"AV. ENG. ABDIAS DE CARVALHO, ENTRE OS N. 1785 ..."
freq,4110,645670,1043634,46618


In [ ]:
infracoes.describe(exclude=[object],datetime_is_numeric=True)

,datainfracao
count,2173805
mean,2020-05-17 15:38:00.579353600
min,2019-01-01 00:00:00
25%,2019-09-04 00:00:00
50%,2020-03-17 00:00:00
75%,2021-02-02 00:00:00
max,2021-12-31 00:00:00


In [ ]:

import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=3)

x0 = np.array(acidentes[(acidentes['data']<'2020-01-01')]['data'].value_counts())
x1 = np.array(acidentes[(acidentes['data']>='2020-01-01')&(acidentes['data']<'2021-01-01')]['data'].value_counts())
x2 = np.array(acidentes[(acidentes['data']>='2021-01-01')]['data'].value_counts())

trace0 = go.Histogram(x=x0,name='Acidentes 2019',nbinsx=20)
trace1 = go.Histogram(x=x1,name='Acidentes 2020',nbinsx=20)
trace2 = go.Histogram(x=x2,name='Acidentes 2021',nbinsx=20)

fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig.append_trace(trace2, 1, 3)
fig.update_layout(barmode='stack',height=800,width=1000,yaxis_title='Frequência')
fig.update_xaxes(title_text="N° de infrações por dia", row=1, col=2)
fig.show()

In [ ]:
x0 = np.array(infracoes[(infracoes['datainfracao']<'2020-01-01')]['datainfracao'].value_counts())
x1 = np.array(infracoes[(infracoes['datainfracao']>='2020-01-01')&(infracoes['datainfracao']<'2021-01-01')]['datainfracao'].value_counts())
x2 = np.array(infracoes[(infracoes['datainfracao']>='2021-01-01')&(infracoes['datainfracao']<'2022-01-01')]['datainfracao'].value_counts())
fig = make_subplots(rows=1, cols=3)

trace0=go.Histogram(x=x0,name='Infrações 2019',nbinsx=20)
trace1=go.Histogram(x=x1,name='Infrações 2020',nbinsx=20)
trace2=go.Histogram(x=x2,name='Infrações 2021',nbinsx=20)  
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig.append_trace(trace2, 1, 3)

fig.update_layout(barmode='stack',height=800,width=1000,yaxis_title='Frequência')
fig.update_xaxes(title_text="N° de infrações por dia", row=1, col=2)
fig.show()

## 1- Qual são as infrações mais frequentes na avenida que apresenta o maior número de acidentes?



### Vamos filtrar os acidentes e infrações que ocorreram na AV GOVERNADOR AGAMENON MAGALHAES que foi o local onde ocorreu o maior numero de acidentes

In [ ]:
df = infracoes
df['Agamenon']= infracoes.localcometimento.str.contains("AGAMENON MAGALHAES")
infracoesagamenon = df.query("Agamenon==True")

In [ ]:
infracoesagamenon['descricaoinfracao'].value_counts().nlargest(n=20)

Transitar em velocidade superior à máxima permitida para o local em até 20% (vinte por cento).          37286
Dirigir o veículo segurando o telefone celular.                                                         17063
Executar operação de conversão à esquerda em locais proibidos pela sinalização.                         11980
Dirigir o veículo manuseando o telefone celular.                                                         5982
Dirigir sem atenção ou sem os cuidados indispensáveis à Segurança.                                       4937
Avançar o sinal vermelho do semáforo.                                                                    4400
Dirigir o veículo utilizando-se de telefone celular.                                                     3259
Transitar na faixa ou via exclusiva regulam.  p/  transp.  públ.  coletivo passag.                       3219
Transitar em velocidade superior à máxima permitida para o local em mais de 20% (vinte por cento) at     3199
Dirigir o 

In [ ]:
df = acidentes
df['Agamenon']= df.endereco.str.contains("AGAMENON MAGALHAES")
acidentessagamenon = df.query("Agamenon==True")

In [ ]:
acidentessagamenon['tipo'].value_counts()

COLISÃO TRASEIRA             297
ABALROAMENTO LONGITUDINAL    213
COLISÃO                      124
ABALROAMENTO TRANSVERSAL      56
COLISÃO LATERAL               31
ATROPELAMENTO                 21
COLISÃO TRANSVERSAL           20
COLISÃO COM CICLISTA          19
COLISÃO FRONTAL               13
ATROPELAMENTO DE PESSOA       10
CHOQUE                         9
SEMÁFORO                       6
CAPOTAMENTO                    6
TOMBAMENTO                     6
ENGAVETAMENTO                  6
CHOQUE OBJETO FIXO             5
CHOQUE VEÍCULO PARADO          4
QUEDA                          3
ACID. DE PERCURSO              1
Name: tipo, dtype: int64

### As infrações mais frequentes são Transitar em velocidade superior à máxima permitida para o local em até 20% (vinte por cento), dirigir o veículo segurando o telefone celular e executar operação de conversão à esquerda em locais proibidos pela sinalização. Podemos ir além e calcular a correlação entre essas infrações e a quantidade de acidentes diárias na Av. Gov. Agamenon Magalhães. 

In [ ]:
# Crie uma lista de descrições de infração
descricoes_infracao  = {'Transitar em velocidade superior à máxima permitida para o local em até 20% (vinte por cento).':'velocidade < 20%',
                       'Dirigir o veículo segurando o telefone celular.':'celular',
                       'Executar operação de conversão à esquerda em locais proibidos pela sinalização.':'Conversão proibida',
                       'Dirigir sem atenção ou sem os cuidados indispensáveis à Segurança.':'Digirir sem atenção',
                       'Avançar o sinal vermelho do semáforo.':'Avançar o sinal',
                       'Transitar na faixa ou via exclusiva regulam.  p/  transp.  públ.  coletivo passag.':'Transitar faixa onibus',
                       'Transitar em velocidade superior à máxima permitida para o local em mais de 20% (vinte por cento) at':'Velocidade >20%',
                       'Dirigir o veículo usando calçado que não se firme nos pés ou que comprometa a utilização dos pedais.':'Calçado inadequado',
                       'Executar operação de conversão à direita em locais proibidos pela sinalização.':'Conversão a direita',
                       'Estacionar o veículo no passeio.':'Estacionamento proibido',                                                                         
                       'Parar o veículo na área de cruzamento de vias, prejudicando a circulação de veículos e pedestres.':'Parar no cruzamento',        
                       'Deixar o passageiro de usar o cinto de segurança, conforme o Art. 65, do CTB.':'Cinto de segurança',                          
                       'Quando o veículo estiver em movimento deixar de manter acesa a luz baixa durante a noite.':'Luz baixa',               
                       'Deixar de deslocar, com antecedência, o veículo para a faixa mais à direita, quando for manobrar.':'Manobra',       
                       'Parar o veículo sobre a faixa de pedestre na mudança de sinal luminoso.':'Parar na faixa',                                  
                       'Deixar de indicar com antecedência, mediante gesto regulamentar de braço ou luz indicadora de direçã':'Deixar de indicar',     
                       'Dirigir o veículo com apenas uma das mãos, exceto quando permitido pelo CTB.':'Dirigir com uma mão',                             
                       'Executar operação de retorno com prejuízo da livre circulação ou da segurança, ainda que em locais p':'Retorno inadequado'}


# Crie uma nova tabela que contém apenas as linhas que correspondem às descrições de infração acima
acidentesav = acidentessagamenon['data'].value_counts()
infracoesav = infracoesagamenon['datainfracao'].value_counts()
df = pd.DataFrame()
df = pd.merge(df, infracoesav, how = 'outer', left_index=True, right_index=True)
df = df.rename(columns={'datainfracao': 'infrações'})
df = pd.merge(df,acidentesav, how = 'outer', left_index=True, right_index=True)
df = df.rename(columns={'data': 'Acidentes'})
for keys in descricoes_infracao.keys():
  df = pd.merge(df, infracoesagamenon[(infracoesagamenon['descricaoinfracao']==keys)]['datainfracao'].value_counts(), how = 'outer', left_index=True, right_index=True)
  df = df.rename(columns={'datainfracao': descricoes_infracao[keys]})
  df = df.fillna(0)



In [ ]:
df.corr(method ='spearman')

,infrações,Acidentes,velocidade < 20%,celular,Conversão proibida,Digirir sem atenção,Avançar o sinal,Transitar faixa onibus,Velocidade >20%,Calçado inadequado,Conversão a direita,Estacionamento proibido,Parar no cruzamento,Cinto de segurança,Luz baixa,Manobra,Parar na faixa,Deixar de indicar,Dirigir com uma mão,Retorno inadequado
infrações,1.000000,0.286833,0.114441,0.709814,0.514420,0.639454,0.634053,0.233810,0.077780,0.586244,0.249073,0.381189,0.565098,0.596883,0.614583,0.428741,0.556101,0.494277,0.586983,0.118422
Acidentes,0.286833,1.000000,0.023953,0.261834,0.033349,0.389942,0.295321,-0.141969,-0.076266,0.277651,-0.176292,0.193377,0.311862,0.177174,0.276368,0.234831,0.252624,0.068997,0.276417,-0.113582
velocidade < 20%,0.114441,0.023953,1.000000,-0.323121,-0.031597,-0.192822,-0.187237,-0.120796,0.707660,-0.299309,-0.225535,-0.012989,-0.294502,-0.186299,-0.192957,-0.121816,-0.235242,-0.165163,-0.214913,-0.123212
celular,0.709814,0.261834,-0.323121,1.000000,0.381922,0.656308,0.601914,0.201485,-0.270540,0.738647,0.411161,0.309782,0.644283,0.650486,0.648155,0.444100,0.596778,0.510435,0.628567,0.146413
Conversão proibida,0.514420,0.033349,-0.031597,0.381922,1.000000,0.215205,0.313724,0.305991,0.016285,0.236891,0.528584,0.138982,0.282929,0.360940,0.234361,0.182047,0.286519,0.348259,0.202037,0.229075
Digirir sem atenção,0.639454,0.389942,-0.192822,0.656308,0.215205,1.000000,0.743100,-0.015839,-0.224886,0.646288,0.029146,0.261925,0.610657,0.588309,0.596138,0.528486,0.639184,0.375094,0.617384,-0.033554
Avançar o sinal,0.634053,0.295321,-0.187237,0.601914,0.313724,0.743100,1.000000,0.035684,-0.212009,0.559815,0.104683,0.257808,0.569911,0.617792,0.611399,0.509779,0.586227,0.493398,0.631989,-0.067635
Transitar faixa onibus,0.233810,-0.141969,-0.120796,0.201485,0.305991,-0.015839,0.035684,1.000000,-0.062185,0.145941,0.426285,0.021980,0.100131,0.142162,0.088536,0.003019,0.085160,0.212647,0.085754,0.226556
Velocidade >20%,0.077780,-0.076266,0.707660,-0.270540,0.016285,-0.224886,-0.212009,-0.062185,1.000000,-0.246771,-0.109482,-0.031691,-0.234512,-0.179577,-0.190652,-0.156928,-0.202473,-0.128345,-0.202491,-0.059698
Calçado inadequado,0.586244,0.277651,-0.299309,0.738647,0.236891,0.646288,0.559815,0.145941,-0.246771,1.000000,0.281868,0.267857,0.532631,0.575348,0.584251,0.407135,0.525352,0.375430,0.553799,0.097904


### Dado o resultado,existe correlação fraca entre os dados de acidentes e infrações. Em relação as infrações mais frequentes Digirir sem atenção, a infração por andar utilizando o celular, Parar na faixa de pedestre, Deixar de deslocar, com antecedência, o veículo para a faixa mais à direita, quando for manobrar,Deixar de manter a luz baixa a noite, Parar no cruzamento, Calçado inadequado e Avançar o sinal também apresentaram correlação fraca com os acidentes.    




## 2 - Tempo , condição da via e conservação são variáveis significativas na ocorrência de acidentes ?



### Para responder essa pergunta podemos usar o teste chi quadrado para verificar se existe evidencia estatistica de independencia das variáveis tempo_clima, condicao_via e conservacao_via.

In [ ]:
acidentesnotnan= acidentes.dropna()
from scipy.stats import chi2_contingency
def testechi2(ds1,ds2) :
  cross = pd.crosstab(ds1, ds2,)
  stat, p, dof, expected = chi2_contingency(cross)
  alpha = 0.05
  print("p value is " + str(p))
  if p <= alpha:  
    print('Rejeita H0')
  else:
    print('Nao Rejeita H0')

In [ ]:
print('Teste  χ2 tempo_clima e conservacao_via :')
testechi2(acidentesnotnan['tempo_clima'],acidentesnotnan['conservacao_via'])
print('Teste  χ2 tempo_clima e condicao_via :')
testechi2(acidentesnotnan['tempo_clima'],acidentesnotnan['condicao_via'])
print('Teste  χ2 condicao_via e conservacao_via :')
testechi2(acidentesnotnan['condicao_via'], acidentesnotnan['conservacao_via'])

Teste  χ2 tempo_clima e conservacao_via :
p value is 0.9713184617731951
Nao Rejeita H0
Teste  χ2 tempo_clima e condicao_via :
p value is 2.253983874947682e-104
Rejeita H0
Teste  χ2 condicao_via e conservacao_via :
p value is 0.7955149139981361
Nao Rejeita H0


### De acordo com os resultados do teste χ2 podemos dizer que não existe diferença estatisticamente significante entre tempo_clima e condicao_via, portanto elas podem ser consideradas como dependentes.



##3- Como as restrições de deslocamento da pandemia afetou o número de acidentes e infrações em 2020?



### Ao observar os histogramas da quantidade diária de acidentes e infrações no ano de 2020 podemos ver que existe uma assimetria acentuada e particulamente no histograma de infrações é possivel enxergar um comportamento bimodal o que pode indicar 2 comportamentos diferentes no periodo de pandemia. Vamos avaliar o numero de infrações por dia no primeiro e segunto semestre de 2020 e comparar com 2019:  

In [ ]:
infracoes20201 = infracoes[(infracoes['datainfracao']>='2020-01-01')&(infracoes['datainfracao']<'2020-04-01')]['datainfracao'].value_counts().sort_values()
infracoes20202 = infracoes[(infracoes['datainfracao']>='2020-04-01')&(infracoes['datainfracao']<'2020-07-01')]['datainfracao'].value_counts().sort_values()
infracoes20203 = infracoes[(infracoes['datainfracao']>='2020-07-01')&(infracoes['datainfracao']<'2020-10-01')]['datainfracao'].value_counts().sort_values()
infracoes20204 = infracoes[(infracoes['datainfracao']>='2020-10-01')&(infracoes['datainfracao']<'2021-01-01')]['datainfracao'].value_counts().sort_values()
infracoes20191 = infracoes[(infracoes['datainfracao']>='2019-01-01')&(infracoes['datainfracao']<'2019-04-01')]['datainfracao'].value_counts().sort_values()
infracoes20192 = infracoes[(infracoes['datainfracao']>='2019-04-01')&(infracoes['datainfracao']<'2019-07-01')]['datainfracao'].value_counts().sort_values()
infracoes20193 = infracoes[(infracoes['datainfracao']>='2019-07-01')&(infracoes['datainfracao']<'2019-10-01')]['datainfracao'].value_counts().sort_values()
infracoes20194 = infracoes[(infracoes['datainfracao']>='2019-10-01')&(infracoes['datainfracao']<'2020-01-01')]['datainfracao'].value_counts().sort_values()

In [ ]:
df1 = infracoes20201.to_frame(name='values')
df1['freq'] = np.linspace(0, 1, infracoes20201.size)
df2 = infracoes20202.to_frame(name='values')
df2['freq'] = np.linspace(0, 1, infracoes20202.size)
df3 = infracoes20203.to_frame(name='values')
df3['freq'] = np.linspace(0, 1, infracoes20203.size)
df4 = infracoes20204.to_frame(name='values')
df4['freq'] = np.linspace(0, 1, infracoes20204.size)


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df1['values'], y=df1['freq'],name='Jan-Fev-Mar',mode="markers"))
fig.add_trace(go.Scatter(x=df2['values'], y=df2['freq'],name='Abr-Mai-Jun',mode="markers"))
fig.add_trace(go.Scatter(x=df3['values'], y=df3['freq'],name='Jul-Ago-Set',mode="markers"))
fig.add_trace(go.Scatter(x=df4['values'], y=df4['freq'],name='Out-Nov-Dez',mode="markers"))
fig.update_layout(title='Infrações 2020',height=800,width=1000,yaxis_title='ECDF',
xaxis_title='N° de infrações por dia')
fig.show()

In [ ]:
df1 = infracoes20191.to_frame(name='values')
df1['freq'] = np.linspace(0, 1, infracoes20191.size)
df2 = infracoes20192.to_frame(name='values')
df2['freq'] = np.linspace(0, 1, infracoes20192.size)
df3 = infracoes20193.to_frame(name='values')
df3['freq'] = np.linspace(0, 1, infracoes20193.size)
df4 = infracoes20194.to_frame(name='values')
df4['freq'] = np.linspace(0, 1, infracoes20194.size)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df1['values'], y=df1['freq'],name='Jan-Fev-Mar',mode="markers"))
fig.add_trace(go.Scatter(x=df2['values'], y=df2['freq'],name='Abr-Mai-Jun',mode="markers"))
fig.add_trace(go.Scatter(x=df3['values'], y=df3['freq'],name='Jul-Ago-Set',mode="markers"))
fig.add_trace(go.Scatter(x=df4['values'], y=df4['freq'],name='Out-Nov-Dez',mode="markers"))
fig.update_layout(title='Infrações 2019',height=800,width=1000,yaxis_title='ECDF',
xaxis_title='N° de infrações por dia')
fig.show()

### Ao visualizar os graficos acima, podemos retirar as seguintes informaçõe sobre o ano de 2020:
### No trimestre Abril, Maio e Junho (auge da pandemia e medidas de restrição de mobilização) 75% do número de infrações registrados por dia é menor ou igual a  1113 o que difere do observado em 2019(2334). 
### Em relação ao último trimestre, temos 75% do número de infrações por dia menor ou igual a 3353 em 2020 e 3333 em 2019. A partir das informações fornecidas, é possível concluir que houve uma significativa queda no número de infrações registradas durante o auge da pandemia em 2020 em comparação com o mesmo período do ano anterior. Isso pode ser atribuído às medidas de restrição de mobilidade implementadas para conter a propagação da COVID-19. Além disso, também é possível ver que o número de infrações no último trimestre de 2020 foi semelhante ao do mesmo período do ano anterior, sugerindo que as medidas de restrição foram relaxadas e o tráfego de veículos voltou ao nível anterior à pandemia.



##4 - Existe têndencia de decrescimo no número de infrações e acidentes?

In [ ]:
#frenquencia diária
seriesfreqacidentes =  acidentes['data'].value_counts()
seriesfreqinfracoes =  infracoes['datainfracao'].value_counts()

In [ ]:
dfacidpordia = seriesfreqacidentes.to_frame()
dfacidpordia = dfacidpordia.reset_index()
dfacidpordia = dfacidpordia.rename(columns={'data': 'acidentes','index':'data'})
dfacidpordia = dfacidpordia.sort_values(by='data')
dfinfrapordia = seriesfreqinfracoes.to_frame()
dfinfrapordia = dfinfrapordia.reset_index()
dfinfrapordia = dfinfrapordia.rename(columns={'datainfracao': 'infracoes','index':'data'})
dfinfrapordia = dfinfrapordia.sort_values(by='data')

In [ ]:
#importar pacotes e setar configurações de plots
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose

dfinfrapordia['data'] = pd.to_datetime(dfinfrapordia['data'])
dfinfrapordia.set_index('data',inplace=True) 
result = seasonal_decompose(dfinfrapordia['infracoes'])
fig = make_subplots(
            rows=4, cols=1,
            subplot_titles=["Série", "Tendência", "Sazonalidade", "Residuos"])
fig.add_trace(go.Scatter(x=result.seasonal.index, y=result.observed, mode='lines'),row=1, col=1,)
fig.add_trace(go.Scatter(x=result.trend.index, y=result.trend, mode='lines'),row=2, col=1,)
fig.add_trace(go.Scatter(x=result.seasonal.index, y=result.seasonal, mode='lines'),row=3, col=1,)
fig.add_trace(go.Scatter(x=result.resid.index, y=result.resid, mode='lines'),row=4, col=1,
)

fig.show()

In [ ]:
dfacidpordia['data'] = pd.to_datetime(dfacidpordia['data'])
dfacidpordia.set_index('data',inplace=True) 
result = seasonal_decompose(dfacidpordia['acidentes'],period=1)
fig = make_subplots(rows=4, cols=1,subplot_titles=["Série", "Tendência", "Sazonalidade", "Residuos"])
fig.add_trace(go.Scatter(x=result.seasonal.index, y=result.observed, mode='lines'),row=1, col=1,)
fig.add_trace(go.Scatter(x=result.trend.index, y=result.trend, mode='lines'),row=2, col=1,)
fig.add_trace(go.Scatter(x=result.seasonal.index, y=result.seasonal, mode='lines'),row=3, col=1,)
fig.add_trace(go.Scatter(x=result.resid.index, y=result.resid, mode='lines'),row=4, col=1,
)

fig.show()

### Podemos vizualizar, através do gráfico de tendencia de acidentes e infrações, que existe um vale que inicia em Março de 2020, em relação as infrações a partir de Junho de 2020 existe um pequeno aumento e permanece desse nivel com flutuações. O nível de acidentes permanece de Março até Dezembro de 2021. É possível que essa tendência de queda nas infrações e acidentes a partir de março de 2020 seja devido às medidas de distanciamento social e lockdowns implementadas em muitos países devido à pandemia de COVID-19. Com menos pessoas nas ruas e menos tráfego de veículos, pode haver menos oportunidades para infrações e acidentes. O aumento leve nas infrações a partir de junho de 2020 pode ser devido ao relaxamento das medidas de distanciamento social e ao aumento do tráfego de veículos. A estabilidade no nível de acidentes até dezembro de 2021 pode ser devido ao equilíbrio entre o aumento do tráfego e as medidas de segurança ainda em vigor.

## 5 - Quais são os veiculos mais importantes em relação ao numero de acidentes com vítimas?

In [ ]:
veiculos = ['auto', 'moto', 'ciclom', 'ciclista', 'pedestre']
acidcomvit = []
acidcomvitf = []
for itens in veiculos:
  y = acidentes[(acidentes[itens]>0)]['vitimas'].sum()
  x = itens
  acidcomvit.append([x,y])
x1,y1=zip(*acidcomvit)
for itens in veiculos:
  y = acidentes[(acidentes[itens]>0)]['vitimasfatais'].sum()
  x = itens
  acidcomvitf.append([x,y])
x2,y2=zip(*acidcomvitf)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=x1,y=y1,mode='markers',name='Vítimas'))
fig.add_trace(go.Scatter(x=x2,y=y2,mode='markers',name='Vítimas fatais'))
fig.update_layout(title='Nº de Vítimas',height=600,width=800,yaxis_title='N° de vitimas',
xaxis_title='Tipo de veículo')
fig.update_traces(mode='markers', marker_line_width=2, marker_size=10)
fig.show()

Verificamos que os acidentes com maior número de vitimas são os que envolvem carros e motos, seguidos pelos acidentes envolvendo pedestres, ciclistas e ciclomotos. Vamos explorar melhor os acidentes envolvendo carros e motos:

In [ ]:

acidentescarro = acidentes[(acidentes['auto']>0)&(acidentes['moto']==0)]
acidentesmoto = acidentes[(acidentes['moto']>0)&(acidentes['auto']==0)]
acidentescarroemoto = acidentes [(acidentes['auto']>0)&(acidentes['moto']>0)]


In [ ]:
acidentescarro.describe(include=[object])

,hora,natureza_acidente,bairro,endereco,tipo,tempo_clima,condicao_via,conservacao_via,velocidade_max_via,mao_direcao
count,683,683,682,681,650,464,463,459,159,453
unique,516,4,73,286,14,3,4,3,13,2
top,10:13:00,COM VÍTIMA,BOA VIAGEM,AV GOVERNADOR AGAMENON MAGALHAES,COLISÃO TRASEIRA,Bom,Seca,Perfeito estado,60 km/h,Dupla
freq,5,339,79,43,132,394,389,439,80,260


In [ ]:
acidentesmoto.describe(include=[object])

,hora,natureza_acidente,bairro,endereco,tipo,tempo_clima,condicao_via,conservacao_via,velocidade_max_via,mao_direcao
count,381,381,381,381,377,325,324,317,128,318
unique,322,4,71,176,14,3,3,4,5,2
top,10:03:00,COM VÍTIMA,BOA VIAGEM,AV NORTE,COLISÃO LATERAL,Bom,Seca,Perfeito estado,60 km/h,Dupla
freq,3,346,25,18,75,274,271,296,78,213


In [ ]:
acidentescarroemoto.describe(include=[object])

,hora,natureza_acidente,bairro,endereco,tipo,tempo_clima,condicao_via,conservacao_via,velocidade_max_via,mao_direcao
count,4687,4690,4675,4684,4678,3891,3887,3833,1241,3787
unique,1163,3,93,794,17,3,4,4,10,3
top,12:15:00,COM VÍTIMA,BOA VIAGEM,AV GOVERNADOR AGAMENON MAGALHAES,ABALROAMENTO LONGITUDINAL,Bom,Seca,Perfeito estado,60 km/h,Dupla
freq,14,3236,529,190,1137,3502,3533,3610,664,2444


In [ ]:
descr = ['Carro','Moto','Carro e Morro']
y11 = acidentescarro['vitimas'].sum()
y12 = acidentesmoto['vitimas'].sum()
y13 = acidentescarroemoto['vitimas'].sum()
y21 = acidentescarro['vitimasfatais'].sum()
y22 = acidentesmoto['vitimasfatais'].sum()
y23 = acidentescarroemoto['vitimasfatais'].sum()
z1  =  len(acidentescarro.index)
z2  =  len(acidentesmoto.index)
z3  =  len(acidentescarroemoto.index)
y1 = [y11,y12,y13]
y2 = [y21,y22,y23]     
z = [z1,z2,z3]

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=descr,y=y1,mode='markers',name='Vítimas fatais',text=['Nº Acidentes 683','Nº Acidentes 381', 'Nº Acidentes 4690']))
fig.add_trace(go.Scatter(x=descr,y=y2,mode='markers',name='Vítimas fatais',text=['Nº Acidentes 683','Nº Acidentes 381', 'Nº Acidentes 4690']))
fig.update_layout(title='Nº de Vítimas',height=600,width=600,yaxis_title='N° de vitimas',
xaxis_title='Tipo de veículo')
fig.update_traces(mode='markers', marker_line_width=2, marker_size=[30,20,50])
fig.show()

### Analisando as ocorrências envolvendo carros e motos separadamente, vemos que apesar do número de acidentes de moto ser menor que o de acidentes de carro (381 acidentes) o número de vítimas chega a 448 o que pode ser justificado pela vulnerabilidade do motociclista no transito. Outro aspecto muito importante que deve ser levado em consideração é a possivel subnotificação de vitimas fatais neste dataset pois considera o número de vítimas fatais momento do acidente e não as que não sobrevivem após o socorro. O maior número de acidentes envolve carros e motos. Em relação ao tipo de acidente, nas ocorrências envolvento apenas motos prevalece a colisão lateral, nos acidentes de carro a frequência de colisão traseira é maior e nos acidentes envolvendo carros e motos o abaloamento longitudinal prevalece em número de ocorrências. 

##6- Existem horários específicos do dia em que as infrações de trânsito ou acidentes são mais prováveis ​​de ocorrer?

In [ ]:
#frenquencia por hora
seriesfreqacidentes =  acidentes2021['hora'].value_counts()
seriesfreqinfracoes =  infracoes2021['horainfracao'].value_counts()
dfacidporhora = seriesfreqacidentes.to_frame()
dfacidporhora = dfacidporhora.reset_index()
dfacidporhora = dfacidporhora.rename(columns={'hora': 'acidentes','index':'hora'})
dfacidporhora = dfacidporhora.sort_values(by='hora')
dfinfraporhora = seriesfreqinfracoes.to_frame()
dfinfraporhora = dfinfraporhora.reset_index()
dfinfraporhora = dfinfraporhora.rename(columns={'horainfracao': 'infracoes','index':'hora'})
dfinfraporhora = dfinfraporhora.sort_values(by='hora')
dfacidporhora['hora'] = pd.to_datetime(dfacidporhora['hora'], format='%H:%M:%S', errors='coerce')
dfacidporhora.dropna(subset=['hora'], inplace=True)
dfacidporhora.set_index('hora', inplace=True)
dfacidporhora = dfacidporhora.resample('H').sum()
dfinfraporhora['hora'] = pd.to_datetime(dfinfraporhora['hora'], format='%H:%M:%S', errors='coerce')
dfinfraporhora.dropna(subset=['hora'], inplace=True)
dfinfraporhora.set_index('hora', inplace=True)
dfinfraporhora =dfinfraporhora.resample('H').sum()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=dfinfraporhora.index, y=dfinfraporhora['infracoes'],mode='markers',name='Infrações'))
fig.update_layout(title='Infrações por hora',height=800,width=1000)
fig.update_traces(mode='markers', marker_line_width=2, marker_size=10)
fig.update_xaxes(tickformat='%H:%M')
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=dfacidporhora.index, y=dfacidporhora['acidentes'],name='Acidentes',mode='markers'))
fig.update_layout(title='Acidentes por hora',height=800,width=1000)
fig.update_traces(mode='markers', marker_line_width=2, marker_size=10)
fig.update_xaxes(tickformat='%H:%M')
fig.show()

Os dados mostram que após as 06:00 as infrações crescem até atingir um picos  as 09:00 e outro as 15:00, após as 21:00 o número de infrações diminui. Os acidentes crescem às 07:00 e seguem com pequenas oscilações até às 20:00. Os horários com maior frequência de acidentes e infrações estão em concordância com o horário comercial de 8:00 às 17:00 levando em consideração as 07:00 como horário de partida para o trabalho e às 21:00 como horário de chegada. Podemos verificar quais são as infrações predominantes no intervalo das 09:00 e das 15:00.

In [ ]:
infracoes09hrs = infracoes2021[(infracoes2021['horainfracao']>='09:00:00')&(infracoes2021['horainfracao']<'10:00:00')]
infracoes15hrs= infracoes2021[(infracoes2021['horainfracao']>='15:00:00')&(infracoes2021['horainfracao']<'16:00:00')]

In [ ]:
infracoes09hrs['descricaoinfracao'].value_counts().nlargest(n=5)

Transitar em velocidade superior à máxima permitida para o local em até 20% (vinte por cento).          12975
Estacionar o veículo em desacordo com as condições regulamentadas - estacionamento rotativo.            11151
Estacionar o veículo em locais e horários proibidos especificamente pela sinalização (Placa - Proibi     7290
Dirigir o veículo segurando o telefone celular.                                                          5367
Estacionar o veículo no passeio.                                                                         3374
Name: descricaoinfracao, dtype: int64

In [ ]:
infracoes09hrs['agenteequipamento'].value_counts().nlargest(n=5)

Código 8 - AUTOS NO TALÃO ELETRÔNICO     31690
Código 3 - LOMBADA ELETRÔNICA            14534
Código 7 - ZONA AZUL TALÃO ELETRÔNICO    12673
Código 5 - FOTO SENSOR                    3150
Código 4 - RADAR                          1425
Name: agenteequipamento, dtype: int64

In [ ]:
infracoes15hrs['descricaoinfracao'].value_counts().nlargest(n=5)

Transitar em velocidade superior à máxima permitida para o local em até 20% (vinte por cento).          15252
Estacionar o veículo em desacordo com as condições regulamentadas - estacionamento rotativo.            13081
Estacionar o veículo em locais e horários proibidos especificamente pela sinalização (Placa - Proibi     7616
Estacionar o veículo no passeio.                                                                         3638
Dirigir o veículo segurando o telefone celular.                                                          3443
Name: descricaoinfracao, dtype: int64

In [ ]:
infracoes15hrs['agenteequipamento'].value_counts().nlargest(n=5)

Código 8 - AUTOS NO TALÃO ELETRÔNICO     27463
Código 3 - LOMBADA ELETRÔNICA            17262
Código 7 - ZONA AZUL TALÃO ELETRÔNICO    14960
Código 5 - FOTO SENSOR                    3189
Código 4 - RADAR                          1175
Name: agenteequipamento, dtype: int64

Segundo dos dados, prevalecem multas de estacionamento indevido e ultrapassagem de velocidade em até 20% além de dirigir o veículo segurando o telefone celular. Podemos inferir que essas são as multas maios comuns no horário comercial.